In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import cv2
from tqdm import tqdm
import random
import shutil

import sys
sys.path.append('../inference/')
from face_detector import FaceDetector

In [ ]:
IMAGES_DIR = '/home/gpu2/hdd/dan/WIDER/WIDER_val/images/'
MODEL_PATH = '../model.pb'

# you will need the content of this folder 
# for the official evaluation script
RESULT_DIR = 'result_wider/'

# Collect paths to images

In [ ]:
all_paths = []
for path, subdirs, files in tqdm(os.walk(IMAGES_DIR)):
    for name in files:
        all_paths.append(os.path.join(path, name))
        
relative_paths = [os.path.relpath(p, IMAGES_DIR) for p in all_paths]

# Predict using the trained detector

In [ ]:
face_detector = FaceDetector(MODEL_PATH, gpu_memory_fraction=0.5, visible_device_list='1')

In [ ]:
predictions = []
for path in tqdm(all_paths):
    image_array = cv2.imread(path)
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    boxes, scores = face_detector(image_array, score_threshold=0.01)
    predictions.append((boxes, scores))

In [ ]:
shutil.rmtree(RESULT_DIR, ignore_errors=True)
os.mkdir(RESULT_DIR)

for (boxes, scores), path in zip(predictions, relative_paths):
    
    assert path.endswith('.jpg')
    file_path = os.path.join(RESULT_DIR, path)[:-4] + '.txt'
    name = path.split('/')[-1][:-4]
    
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    with open(file_path, 'w') as f:
        
        f.write(name + '\n')
        f.write(str(len(boxes)) + '\n')
        
        for b, s in zip(boxes, scores):
            ymin, xmin, ymax, xmax = b
            h, w = int(ymax - ymin), int(xmax - xmin)
            f.write('{0} {1} {2} {3} {4:.4f}\n'.format(int(xmin), int(ymin), w, h, s))